
SIIM-ACR Pneumothorax Segmentation 

Infernce kernel for Unet Resnet 34 

## Import Libraries 

In [0]:
import os
import cv2
import pdb
import time
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.torch import ToTensor
from torchvision import transforms

 
from matplotlib.pyplot import figure
warnings.filterwarnings("ignore")

## Segmentation Models

In [0]:
! pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [0]:
import segmentation_models_pytorch as smp

## Global Variables 

In [0]:
IMAGE_SIZE =1024
BATCH_SIZE = 2
EPOCH = 65

## Utility functions

In [0]:
def run_length_decode(rle, height=1024, width=1024, fill_value=1):
    component = np.zeros((height, width), np.float32)
    component = component.reshape(-1)
    rle = np.array([int(s) for s in rle.strip().split(' ')])
    rle = rle.reshape(-1, 2)
    start = 0
    for index, length in rle:
        start = start+index
        end = start+length
        component[start: end] = fill_value
        start = end
    component = component.reshape(width, height).T
    return component

def run_length_encode(component):
    component = component.T.flatten()
    start = np.where(component[1:] > component[:-1])[0]+1
    end = np.where(component[:-1] > component[1:])[0]+1
    length = end-start
    rle = []
    for i in range(len(length)):
        if i == 0:
            rle.extend([start[0], length[0]])
        else:
            rle.extend([start[i]-end[i-1], length[i]])
    rle = ' '.join([str(r) for r in rle])
    return rle

In [0]:
sample_submission_path = '../input/siim-acr-pneumothorax-segmentation/stage_2_sample_submission.csv'
train_rle_path = '../input/siim-dicom-images/train-rle.csv'
data_folder = "../input/siim-png-images/input/train_png"
#test_data_folder = "../input/siim-png-images/input/test_png"
test_data_folder = "../input/stage2-test-images/"


## Some more utility functions

Here are some utility functions for calculating IoU and Dice scores

In [0]:
def predict(X, threshold):
    X_p = np.copy(X)
    preds = (X_p > threshold).astype('uint8')
    return preds

def metric(probability, truth, threshold=0.5, reduction='none'):
    '''Calculates dice of positive and negative images seperately'''
    '''probability and truth must be torch tensors'''
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = (truth > 0.5).float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = torch.cat([dice_pos, dice_neg])

        dice_neg = np.nan_to_num(dice_neg.mean().item(), 0)
        dice_pos = np.nan_to_num(dice_pos.mean().item(), 0)
        dice = dice.mean().item()

        num_neg = len(neg_index)
        num_pos = len(pos_index)

    return dice, dice_neg, dice_pos, num_neg, num_pos

class Meter:
    '''A meter to keep track of iou and dice scores throughout an epoch'''
    def __init__(self, phase, epoch):
        self.base_threshold = 0.5 # <<<<<<<<<<< here's the threshold
        self.base_dice_scores = []
        self.dice_neg_scores = []
        self.dice_pos_scores = []
        self.iou_scores = []

    def update(self, targets, outputs):
        probs = torch.sigmoid(outputs)
        dice, dice_neg, dice_pos, _, _ = metric(probs, targets, self.base_threshold)
        self.base_dice_scores.append(dice)
        self.dice_pos_scores.append(dice_pos)
        self.dice_neg_scores.append(dice_neg)
        preds = predict(probs, self.base_threshold)
        iou = compute_iou_batch(preds, targets, classes=[1])
        self.iou_scores.append(iou)

    def get_metrics(self):
        dice = np.mean(self.base_dice_scores)
        dice_neg = np.mean(self.dice_neg_scores)
        dice_pos = np.mean(self.dice_pos_scores)
        dices = [dice, dice_neg, dice_pos]
        iou = np.mean(self.iou_scores)
        return dices, iou

def epoch_log(phase, epoch, epoch_loss, meter, start):
    '''logging the metrics at the end of an epoch'''
    dices, iou = meter.get_metrics()
    dice, dice_neg, dice_pos = dices
    print("dice: %0.4f | dice_neg: %0.4f | dice_pos: %0.4f | IoU: %0.4f" % (dice, dice_neg, dice_pos, iou))
    return None

def compute_ious(pred, label, classes, ignore_index=255, only_present=True):
    '''computes iou for one ground truth mask and predicted mask'''
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c, label_c).sum()
        union = np.logical_or(pred_c, label_c).sum()
        if union != 0:
            ious.append(intersection / union)
    return ious if ious else [1]


def compute_iou_batch(outputs, labels, classes=None):
    '''computes mean iou for a batch of ground truth masks and predicted masks'''
    ious = []
    preds = np.copy(outputs) # copy is imp
    labels = np.array(labels) # tensor to np
    for pred, label in zip(preds, labels):
        ious.append(np.nanmean(compute_ious(pred, label, classes)))
    iou = np.nanmean(ious)
    return iou


## UNet with ResNet34 
 

In [0]:
model1 = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)
model2 = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)
model3 = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)
model4 = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)
model5 = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)
model6 = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)
 
 

model1.cuda()
model2.cuda()
model3.cuda()
model4.cuda()
model5.cuda()
model6.cuda()
 


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /tmp/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 112MB/s]


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

## Test prediction

In [0]:
class TestDataset(Dataset):
    def __init__(self, root, df, size, mean, std, tta=4):
        self.root = root
        self.size = size
       
        self.mean = mean
        self.std = std
        self.fnames = list(df["ImageId"])
        self.num_samples = len(self.fnames)
        self.transforms = get_transforms(size, mean, std)

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname + ".png")
        image = cv2.imread(path)
        augmented = self.transforms(image=image)
        image = augmented['image']
       
        return image

    def __len__(self):
        return self.num_samples
    
    
    
def get_transforms(size, mean, std):
        list_transforms = []
        
        list_transforms.extend(
        [
            Normalize(mean=mean, std=std, p=1),
            Resize(size, size),
            ToTensor(),
        ]
    )
        
        list_trfms = Compose(list_transforms)
        return list_trfms

def post_process(probability, threshold, min_size):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((1024, 1024), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [0]:
size = IMAGE_SIZE
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
num_workers = 8
batch_size = 1
best_threshold = 0.5
min_size = 3500
device = torch.device("cuda:0")
df = pd.read_csv(sample_submission_path)
testset = DataLoader(
    TestDataset(test_data_folder, df, size, mean, std),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)



#model = model_trainer.net # get the model from model_trainer object



model1.eval()
state = torch.load('../input/unetresnet34-run3-04-09/UnetResnet34_Run3__04_09.pth', map_location=lambda storage, loc: storage)
model1.load_state_dict(state["state_dict"])


model2.eval()
state = torch.load('../input/unet-resnet34-01-09/Unet_resnet34_run2_01_09.pth', map_location=lambda storage, loc: storage)
model2.load_state_dict(state["state_dict"])



model3.eval()
state = torch.load('../input/unetresnet34-stage2-run3-02-09/UnetTesnet34_Run3_Stage2_02_09.pth', map_location=lambda storage, loc: storage)
model3.load_state_dict(state["state_dict"])


model4.eval()
state = torch.load('../input/unetresn34-run3-02-09-v2/Unetresnet34_run3_V2.pth', map_location=lambda storage, loc: storage)
model4.load_state_dict(state["state_dict"])


model5.eval()
state = torch.load('../input/unet-resnet34-stage2-run2-31-08/Unet_Resnet34_Run2_Stage2_31_08.pth', map_location=lambda storage, loc: storage)
model5.load_state_dict(state["state_dict"])


model6.eval()
state = torch.load('../input/unetresnet34-run3-05-09/model.pth', map_location=lambda storage, loc: storage)
model6.load_state_dict(state["state_dict"])


for param in model1.parameters():
    param.requires_grad = False


for param in model2.parameters():
    param.requires_grad = False

for param in model3.parameters():
    param.requires_grad = False

for param in model4.parameters():
    param.requires_grad = False
    
for param in model5.parameters():
    param.requires_grad = False
    
for param in model6.parameters():
    param.requires_grad = False


encoded_pixels = []
#dataloader_iterator = iter(testset2)
for i, batch in enumerate(tqdm(testset)):
   
   
    preds = torch.sigmoid(model1(batch.to(device)))
    preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    
   
    preds2 = torch.sigmoid(model2(batch.to(device)))
    preds2 = preds2.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    
    preds3 = torch.sigmoid(model3(batch.to(device)))
    preds3 = preds3.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
   
    preds4 = torch.sigmoid(model4(batch.to(device)))
    preds4 = preds4.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    
    preds5 = torch.sigmoid(model5(batch.to(device)))
    preds5 = preds5.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
   
    preds6 = torch.sigmoid(model6(batch.to(device)))
    preds6 = preds6.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
   
    
    preds = ( preds+ preds2 + preds3 + preds4 +  preds6) / 5.0 
  
    for probability in preds:
        if probability.shape != (1024, 1024):
            probability = cv2.resize(probability, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)
        predict, num_predict = post_process(probability, best_threshold, min_size)
        if num_predict == 0:
            encoded_pixels.append('-1')
        else:
            r = run_length_encode(predict)
            encoded_pixels.append(r)
df['EncodedPixels'] = encoded_pixels
df.to_csv('submission.csv', columns=['ImageId', 'EncodedPixels'], index=False)